In [ ]:
# Imports
import ipywidgets as widgets
import threading
from Trainer import DQNtrainer
from System import System

# Test-System

## Systemaufbau

Das Test-System besteht aus einer nichtlinearen Regelstrecke (PT1-Glied). Deren Ausgang wird durch eine Störung (ebenfalls verzögert durch ein PT1-Glied) beeinflusst. Geregelt werden soll die Ausgangsgröße $y$ des Gesamtsystems. Die relevanten Übertragungsfunktionen lauten:

$$ F_S = \frac{y_S}{u_S}= \frac{K(y_S)}{1 + T_Sp} $$
$$ F_Z = \frac{y_Z}{u_Z} = \frac{1}{1 + 10T_Sp} $$
mit
\begin{align}
    K(y_S) &= 0.1\cdot y_S + 2 \\
    -5 &\leq u_S \leq 5 \\
    -2 &\leq u_Z \leq 2 \\
    y &= y_S + y_Z \\
    -8.5 &\leq y \leq 21.5
\end{align}

Folgende größen werden als messbar angenommen:
- $y$
- $y_S$
- $u_S$
- $u_Z$

<img src="../SFB.png" alt="Signalflussbild des Gesamtsystems" title="Signalflussbild des Gesamtsystems" />

### Herleitung der Solver-Gleichung
Z-Transformation der Übertragungsfunktion für PT1-Glieder:
\begin{align}
    F(p) = \frac{y}{u}= \frac{K}{1 + T\cdot p} \\
    F(z) = \frac{y_k}{u_k} = K \cdot \frac{1-a}{z-a}
\end{align}
mit
$$a = e^{-\frac{dt}{T}}$$

Auflösen nach $y_k$:
\begin{align}
    F(z) = \frac{y_k}{u_k} &= \frac{K-Ka}{z-a} \\
    y_k \cdot (z - a) &= u_k \cdot (K - Ka) \\
    y_{k+1} - ay_k  &= u_k \cdot (K - Ka) \\
    y_{k+1} - ay_k  &= u_k \cdot (K - Ka) \\
    y_{k+1} &= u_k \cdot (K - Ka) + ay_k
\end{align}

In [ ]:
# Prepare test system
system = System(0.0)

# Regler
Der selbstlernende Regler wird mittels Deep-Q-Learning Realisiert. Die Implementierung erfolgt in Anlehnung an das DQN Tutorial von pytorch sowie an das entsprechende DeepMind Paper (Lernen von Atari Spielen). Das DQN verfahren ist anders als NFQ lernen ein online Verfahren. Es scheint die Weiterentwicklung des NFQ Verfahrens zu sein und zielt auf eine bessere konvergenz der zu trainierenden Modelle ab. Diese ist insbesondere bei großen KNN von Bedeutung. Daher wurde von DeepMind das Bsp. Atari-Spiele gewählt. Wobei der Input die Pixel-Daten der Spielegrafik waren und entsprechende Faltungsschichten selbstständig die wesentlichen merkmale bestimmen mussten. Da zeitlich korellierte Daten schlecht für das Training sind, werden eine Vielzahl an Datensätzen in einem Speicher (Replay-Memory) abgelegt. Zu jedem Prozessschritt werden eine kleine Anzahl an zufälligen Datenpunkten aus dem Speicher gezogen und das Modell trainiert. Dabei wird das Modell doppelt verwendet. Einmal als in jedem Schritt trainiertes und für die Handlungen konsultierte Modell. Das andere soll Q-werte, welche für das Training als Referenzwerte dienen, vorhersagen. Letzteres wird nur zu wenigen Schritten verbessert. Dabei werden die aktuellen Parameter (Gewichte, etc.) des anderen Modells kopiert.

Ein System-Zustand ist durch die Messbaren signale gekennzeichnet: $w, y, y_S, u_S, u_Z$

## Agent mit Erfahrungsspeicher und -modell

In [ ]:
# hyper parameter
nBatch = 100
batchSize = 32
probW = 0.5 # probability for change of set point

memCapacity = 320000

eps0 = 1.
epsDecay = 500
epsMin = 0.0

targetUpdate = 250  # rate of epochs for updating target model

# max cost per action
cMax = 0.1
mu = 0.5  # target range for cost function

# init trainer with agent
trainer = DQNtrainer(nBatch, batchSize, targetUpdate,
                     eps0, epsMin, epsDecay, cMax, mu, system,
                     memCapacity)


In [ ]:
# Control Elements
btnRun = widgets.Button(description="Run")
btnRun.state = False
btnEnd = widgets.Button(description="End")
btnEnd.state = True
btnStep = widgets.Button(description="Step")
btnStep.state = False
btnReset = widgets.Button(description="Reset")
btnReset.state = False

def btnRun_click(btn):
    if btn.state:
        btn.state = False
        btn.description = "Run"
    else:
        btn.state = True
        btn.description = "Pause"

def btn_click(btn):
    btn.state = not btn.state

btnRun.on_click(btnRun_click)
btnEnd.on_click(btn_click)
btnStep.on_click(btn_click)
btnReset.on_click(btn_click)


## Training

In [ ]:
buttons = widgets.HBox([btnRun, btnStep, btnReset, btnEnd])
display(widgets.VBox([buttons, trainer.batchVis, trainer.trainVis]))
thread = threading.Thread(target=trainer.train, args=(system, btnRun, btnStep, 
                                                      btnEnd, btnReset,))
thread.start()